In [16]:
# Only operate when the newjoblist need to be resetted
import csv
with open('newjoblist.csv','w',newline='') as g:
    writer = csv.writer(g)
    header = ['Title','Company','Location','Date','URL']
    writer.writerow(header)

In [1]:
%%time
import requests
from bs4 import BeautifulSoup
import csv

def scrape_one_page(page_url):
    jobs= []
    r = requests.get(page_url).text
    data = BeautifulSoup(r,"html.parser")
    if page_url.find('indeed')!=-1:
        all_h2 = data.find_all('h2', attrs={'class':"jobtitle"}) 
        #it's a weirdo page that the last item's 'class' is different from above 9, so that we use its sub label h2.
        for i in all_h2:
            job = []
            job.append(i.a['title'])
            job.append(i.parent.find('span', attrs={'class':"company"}).text.strip()) 
            #use .parent back to the higher label
            job.append(i.parent.find('span', attrs={'class':"location"}).text.strip())
            job.append(i.parent.find('span', attrs={'class':"date"}).text.strip())
            job.append('https://www.indeed.com/viewjob?jk={}'.format(i['id'][3:]))
            jobs.append(job)
    
    elif page_url.find('careers')!=-1:
        all_div = data.find_all('div', attrs={'class':"bti-ui-job-detail-container"}) 
        for i in all_div:
            job=[]
            job.append(i.find('a').text)
            job.append(i.find('div', attrs={'class':"bti-ui-job-result-detail-employer"}).text.strip())
            job.append(i.find('div', attrs={'class':"bti-ui-job-result-detail-location"}).text.strip())
            job.append(i.find('div', attrs={'class':"bti-ui-job-result-detail-age"}).text.strip())
            job.append('https://careers.journalists.org{}'.format(i.find('a')['href']))
            jobs.append(job)

    elif page_url.find('jobsdb')!=-1:
        all_div = data.find_all('div', attrs={'class':"_3ASfTyv _2EUSthc"}) 
        for i in all_div:
            job=[]
            job.append(i.find('div', attrs={'class':"_3gfm7U9 _3ho-Knb _2swcdgn"}).a.text)
            job.append(i.find('div', attrs={'class':"_1NdWRqw _3ho-Knb _2swcdgn"}).find('span').text)
            job.append(i.find('div', attrs={'class':"_124cxoK _3ho-Knb _2swcdgn"}).find('span').text)
            job.append(i.find('span', attrs={'class':"JG37Vx2 _3Re95QG _2XGgj_O"}).find('span').text)
            job.append(i.find('div', attrs={'class':"_3gfm7U9 _3ho-Knb _2swcdgn"}).a['href'])
            jobs.append(job)
    return jobs

def scrape_all_pages(base_url,step):
    all_jobs=[]
    level = 0
    while True: 
        level += step  #level is in url to indiccate different pages' index
        page_url = '{}{}'.format(base_url,level)
        jobs = scrape_one_page(page_url)
        if jobs == []: #if all jobs have been scraped, break the loop
            break
        all_jobs.extend(jobs)
        if len(all_jobs) > 100: #only scrape 100 jobs from each website
            break
    return all_jobs

def output_new_jobs(website):
    with open('existedjoblist_{}.csv'.format(website),'r') as f:
        myreader = csv.reader(f)
        url_existed = [row[4] for row in myreader] #read a row

    with open('newjoblist.csv','a') as g:
        mywriter = csv.writer(g)
        for i in all_jobs:
            if i[4] not in url_existed: #duplicate checking
                mywriter.writerow(i)
            
    with open('existedjoblist_{}.csv'.format(website),'w') as h:
        mywriter=csv.writer(h)
        header=['Title','Company','Location','Date','URL']
        mywriter.writerow(header) 
        mywriter.writerows(all_jobs)

websites = [
    {
        "website": 'indeed',
        "base_url": 'https://www.indeed.com/jobs?q=Data+Journalist+Internship&start=',
        "step": 10,
    },
    {
        "website": 'careers',
        "base_url": 'https://careers.journalists.org/jobs/?keywords=data+OR+journalist+OR+internship&page=',
        "step": 1,
    }, 
    {
        "website": 'jobsdb',
        "base_url": 'https://hk.jobsdb.com/hk/search-jobs/data-journalist/',
        "step": 1,
    },
]  

for i in websites:
    all_jobs = scrape_all_pages(i["base_url"],i["step"])
    output_new_jobs(i["website"])

CPU times: user 1.79 s, sys: 77.5 ms, total: 1.87 s
Wall time: 24 s


In [20]:
#just a example, it's not real-time data
import pandas
df=pandas.read_csv('newjoblist.csv')
df[['Title','Company','Location','Date']]

,Title,Company,Location,Date
0,"Symitar Technical Writer, Summer 2019 Internship","Jack Henry and Associates, Inc.","San Diego, CA",16 hours ago
1,Product Documentation Intern,Sony Interactive Entertainment PlayStation,"San Mateo, CA",1 day ago
2,"Symitar Technical Writer, Summer 2019 Internship",Jack Henry,"San Diego, CA 92123",14 hours ago
3,Digital Platforms Intern,Detroit Public Television,"Wixom, MI 48393",15 hours ago
4,Communications Intern,United Nations Development Programme (UNDP),"New York, NY",6 hours ago
5,"Intern Sales Specialty/eCommerc - Phoenix, AZ",FedEx Services,"Phoenix, AZ",1 day ago
6,Digital Communications Intern,Akamai,"Cambridge, MA 02139 (Area IV area)",2 days ago
7,Staff Consultant - Public Affairs Specialist (...,ERM Group,"Richmond, VA",1 day ago
8,Marketing Coordinator - San Luis Obispo,"Water Systems Consulting, Inc.","San Luis Obispo, CA 93403",14 days ago
9,"Assistant Director, Career Services",Columbia University,"New York, NY",18 hours ago
